In [4]:
%cd /content/drive/MyDrive/logicalErrorFix-2

/content/drive/MyDrive/logicalErrorFix-2


In [10]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

from __future__ import absolute_import
import os
import sys
import bleu
import pickle
import torch
import json
import random
import logging
import argparse
import numpy as np
import copy
from io import open
from itertools import cycle
import torch.nn as nn
from model import Seq2Seq
from tqdm import tqdm, trange
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaModel, RobertaTokenizer)

from transformers import T5ForConditionalGeneration
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer),
                 'codeT5' : (None, T5ForConditionalGeneration, RobertaTokenizer)}

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

class Example(object):
    """A single training/test example."""
    def __init__(self,
                 idx,
                 source,
                 target,
                 ):
        self.idx = idx
        self.source = source
        self.target = target

import pandas as pd
COLUMNS = ['Correct_code', 'Incorrect_code', 'Statement']

def read_examples(filename):
  """Read examples from filename for DeepFix style training Line stmt Line stmt Line stmt ..."""
  examples = []
  data = pd.read_csv(filename, sep='\t', header=[0]).drop(columns=COLUMNS[0])
  for idx, elem in data.iterrows():
    code = ' '.join(elem[COLUMNS[1]].split('||| '))[:-1].strip()
    stmt = elem[COLUMNS[2]].strip()

    examples.append(
      Example(
              idx = idx,
              source = code,
              target = stmt,
              )
    )
  return examples

class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask


def process_code_examples(input_examples):
    def parse_and_continue_on_mismatch(code_str):
        code_dict = {}
        expected_line_number = 1
        parts = code_str.split()
        current_line = None
        for part in parts:
            if part.isdigit():
                line_number = int(part)
                if line_number == expected_line_number:
                    current_line = line_number
                    code_dict[current_line] = ""
                    expected_line_number += 1
            else:
                if current_line is not None:
                    code_dict[current_line] += part + " "
        for line in code_dict:
            code_dict[line] = code_dict[line].strip()
        return code_dict

    def remove_line_numbers(source_dict):
        ret = []
        for _, v in source_dict.items():
            ret.append(v)
        return ' '.join(ret)

    def parse_single_line_to_dict(line_str):
        parts = line_str.split(' ', 1)
        line_number = parts[0]
        code = parts[1] if len(parts) > 1 else ""
        return {int(line_number): code}

    def update_values_from_dict(dict1, dict2):
        for key in dict1:
            if key in dict2:
                dict1[key] = dict2[key]
        return dict1

    source_code_dict = {}
    for i, example in enumerate(input_examples):
        source_code_dict[i] = parse_and_continue_on_mismatch(example.source)

    target_dict = {}
    for i, example in enumerate(input_examples):
        target_dict[i] = parse_single_line_to_dict(example.target)

    target_code_dict = {i: update_values_from_dict(copy.deepcopy(source_code_dict[i]), target_dict[i]) for i in range(len(target_dict))}

    input_code = {}
    for i in range(len(source_code_dict)):
        input_code[i] = {
            "source": remove_line_numbers(source_code_dict[i]),
            "target": remove_line_numbers(target_code_dict[i])
        }

    return input_code


def convert_examples_to_features(examples, tokenizer, args,stage=None):
    if args.model_name_or_path != "Salesforce/codet5-base":
        features = []
        for example_index, example in enumerate(examples):
            #source
            source_tokens = tokenizer.tokenize(example.source)[:args.max_source_length-2]
            source_tokens =[tokenizer.cls_token]+source_tokens+[tokenizer.sep_token]
            source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
            source_mask = [1] * (len(source_tokens))
            padding_length = args.max_source_length - len(source_ids)
            source_ids+=[tokenizer.pad_token_id]*padding_length
            source_mask+=[0]*padding_length

            #target
            if stage=="test":
                target_tokens = tokenizer.tokenize("None")
            else:
                target_tokens = tokenizer.tokenize(example.target)[:args.max_target_length-2]
            target_tokens = [tokenizer.cls_token]+target_tokens+[tokenizer.sep_token]
            target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
            target_mask = [1] *len(target_ids)
            padding_length = args.max_target_length - len(target_ids)
            target_ids+=[tokenizer.pad_token_id]*padding_length
            target_mask+=[0]*padding_length

            if example_index < 5:
                ##NNNN
                if stage=='train1':
                    logger.info("*** Example ***")
                    logger.info("idx: {}".format(example.idx))

                    logger.info("source_tokens: {}".format([x.replace('\u0120','_') for x in source_tokens]))
                    logger.info("source_ids: {}".format(' '.join(map(str, source_ids))))
                    logger.info("source_mask: {}".format(' '.join(map(str, source_mask))))

                    logger.info("target_tokens: {}".format([x.replace('\u0120','_') for x in target_tokens]))
                    logger.info("target_ids: {}".format(' '.join(map(str, target_ids))))
                    logger.info("target_mask: {}".format(' '.join(map(str, target_mask))))

            features.append(
                InputFeatures(
                    example_index,
                    source_ids,
                    target_ids,
                    source_mask,
                    target_mask,
                )
            )
    else:
        max_length = 512
        input_code = process_code_examples(examples)
        features = []

        for example_index in range(len(input_code)):
            source = input_code[example_index]["source"]
            input_hf = tokenizer(source, padding="max_length", truncation=True, max_length=max_length)
            source_ids = input_hf.input_ids
            source_mask = input_hf.attention_mask

            target = input_code[example_index]["target"]
            target_hf = tokenizer(target, padding="max_length", truncation=True, max_length=max_length)
            target_ids = target_hf.input_ids
            target_mask = target_hf.attention_mask

            features.append(
                InputFeatures(
                    example_index,
                    source_ids,
                    target_ids,
                    source_mask,
                    target_mask,
                )
            )
    return features


def set_seed(args):
    """set random seed."""
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def main():
    parser = argparse.ArgumentParser()

    ## Required parameters
    parser.add_argument("--model_type", default=None, type=str, required=True,
                        help="Model type: e.g. roberta")
    parser.add_argument("--model_name_or_path", default=None, type=str, required=True,
                        help="Path to pre-trained model: e.g. roberta-base" )
    parser.add_argument("--output_dir", default=None, type=str, required=True,
                        help="The output directory where the model predictions and checkpoints will be written.")
    parser.add_argument("--load_model_path", default=None, type=str,
                        help="Path to trained model: Should contain the .bin files" )
    ## Other parameters
    parser.add_argument("--train_filename", default=None, type=str,
                        help="The train filename. Should contain the .jsonl files for this task.")
    parser.add_argument("--dev_filename", default=None, type=str,
                        help="The dev filename. Should contain the .jsonl files for this task.")
    parser.add_argument("--test_filename", default=None, type=str,
                        help="The test filename. Should contain the .jsonl files for this task.")

    parser.add_argument("--config_name", default="", type=str,
                        help="Pretrained config name or path if not the same as model_name")
    parser.add_argument("--tokenizer_name", default="", type=str,
                        help="Pretrained tokenizer name or path if not the same as model_name")
    parser.add_argument("--max_source_length", default=64, type=int,
                        help="The maximum total source sequence length after tokenization. Sequences longer "
                             "than this will be truncated, sequences shorter will be padded.")
    parser.add_argument("--max_target_length", default=32, type=int,
                        help="The maximum total target sequence length after tokenization. Sequences longer "
                             "than this will be truncated, sequences shorter will be padded.")

    parser.add_argument("--do_train", action='store_true',
                        help="Whether to run training.")
    parser.add_argument("--do_eval", action='store_true',
                        help="Whether to run eval on the dev set.")
    parser.add_argument("--do_test", action='store_true',
                        help="Whether to run eval on the dev set.")
    parser.add_argument("--do_lower_case", action='store_true',
                        help="Set this flag if you are using an uncased model.")
    parser.add_argument("--no_cuda", action='store_true',
                        help="Avoid using CUDA when available")

    parser.add_argument("--train_batch_size", default=8, type=int,
                        help="Batch size per GPU/CPU for training.")
    parser.add_argument("--eval_batch_size", default=8, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
    parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
    parser.add_argument("--learning_rate", default=5e-5, type=float,
                        help="The initial learning rate for Adam.")
    parser.add_argument("--beam_size", default=10, type=int,
                        help="beam size for beam search")
    parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                        help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
    parser.add_argument("--num_train_epochs", default=3.0, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
    parser.add_argument("--eval_steps", default=-1, type=int,
                        help="")
    parser.add_argument("--train_steps", default=-1, type=int,
                        help="")
    parser.add_argument("--warmup_steps", default=0, type=int,
                        help="Linear warmup over warmup_steps.")
    parser.add_argument("--local_rank", type=int, default=-1,
                        help="For distributed training: local_rank")
    parser.add_argument('--seed', type=int, default=42,
                        help="random seed for initialization")
    # print arguments
    args = parser.parse_args()
    logger.info(args)
    import argparse

# 명령줄 인자를 직접 코드 내에서 설정
args = argparse.Namespace(
    model_type='roberta',
    model_name_or_path='roberta-base',
    output_dir='path/to/output/dir',
    load_model_path='path/to/trained/model',
    train_filename='./data/edit_distance/pair_solution_train.txt',
    dev_filename='path/to/dev.jsonl',
    test_filename='path/to/test.jsonl',
    config_name='',
    tokenizer_name='',
    max_source_length=64,
    max_target_length=32,
    do_train=True,
    do_eval=True,
    do_test=True,
    do_lower_case=True,
    no_cuda=True,
    train_batch_size=8,
    eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    beam_size=10,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    num_train_epochs=3.0,
    max_steps=-1,
    eval_steps=-1,
    train_steps=-1,
    warmup_steps=0,
    local_rank=-1,
    seed=42,
)

# 예제 사용
print(args.model_type)  # 출력: roberta
print(args.do_train)    # 출력: True


roberta
True


In [18]:
data = pd.read_csv('./data/edit_distance/pair_solution_train.txt', sep='\t')

In [19]:
print(data.columns)

Index(['6', '524', '610', '5'], dtype='object')
